## Summary

Calculate features using [AlphaFold](https://github.com/deepmind/alphafold).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_alphafold.ipynb)"

export DATASET_NAME="cagi6-sherloc"
export DATASET_PATH="30_cagi6_sherloc/input-data-gby-protein.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=4182

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-3500 --time 24:00:00 --gres=gpu:t4:1 ../scripts/run_notebook_gpu.sh

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=181-200 --time 24:00:00 --gres=gpu:v100:1 ../scripts/run_notebook_gpu.sh

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=201-306 ../scripts/run_notebook_cpu.sh
```

---

## Imports

In [1]:
import concurrent.futures
import os
import re
import subprocess
import sys
import tempfile
from pathlib import Path

import jax
import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.alphafold import AlphaFold, AlphaFoldAnalyzeError, AlphaFoldBuildError
from jax.lib import xla_bridge
from kmbio import PDB
from tqdm.notebook import tqdm

## Parameters

In [2]:
NOTEBOOK_DIR = Path("31_run_alphafold").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold')

In [3]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir
    
print(tempfile.gettempdir())

/tmp


In [4]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

(None, None, None, None)

In [5]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "cagi6-sherloc"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
    )
    TASK_ID = 2000
    TASK_COUNT = 4182
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()
    assert TASK_ID is not None
    assert TASK_COUNT is not None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('cagi6-sherloc',
 '/scratch/strokach/workspace/elaspic2-cagi6/notebooks/30_cagi6_sherloc/input-data-gby-protein.parquet',
 2000,
 4182)

In [6]:
device = xla_bridge.get_backend().platform

device

'gpu'

## Workspace

### Initialize model

In [7]:
AlphaFold.load_model(device=device)

### Load data

In [8]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

4182

In [9]:
assert TASK_COUNT == pfile.num_row_groups

In [10]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [11]:
display(input_df.head(2))
print(len(input_df))

,protein_id,mutation_id,mutation,effect,sequence,structure,alignment
0,P08236,"[NM_000181.3:c.13T>C, NM_000181.3:c.22G>T, NM_...","[S5P, A8S, D53N, L255Q, Q306H, A330S, P364L, S...","[Likely benign, Uncertain significance, Uncert...",MARGSAVAWAALGPLLWGCALGLQGGMLYPQESPSRECKELDGLWS...,HEADER ...,"[>101\n, MARGSAVAWAALGPLLWGCALGLQGGMLYPQESPSRE..."


1


In [12]:
tup = next(input_df.itertuples(index=False))

iterable_fields = []
for field in tup._fields:
    try:
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError:
        pass

iterable_fields

['mutation_id', 'mutation', 'effect']

In [13]:
if DEBUG:
    for field in iterable_fields:
        input_df[field] = input_df[field].str[:3]

### Score mutations

In [14]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None:
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False

    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False

    return True

In [15]:
def worker(mutation, data):
    try:
        results = AlphaFold.analyze_mutation(f"A_{mutation}", data)
    except AlphaFoldAnalyzeError as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"alphafold_core_{key}": value for key, value in results.items()}
        return results

In [16]:
results = []
for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

    data = AlphaFold.build(tup.sequence, ligand_sequence=None, msa=tup.alignment)

    for mutation_idx, mutation in enumerate(tqdm(tup.mutation, leave=False)):
        if not validate_mutation(mutation):
            continue

        result = worker(mutation, data)
        if result is None:
            continue

        results.append(
            {
                "protein_id": tup.protein_id,
                "mutation": mutation,
            }
            | {field: getattr(tup, field)[mutation_idx] for field in iterable_fields}
            | result
        )

results_df = pd.DataFrame(results)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp4bx2qlof.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp4bx2qlof.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp4bx2qlof.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpdjhobk09.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpdjhobk09.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpdjhobk09.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [17]:
display(results_df.head(2))
print(len(results_df))

,protein_id,mutation,mutation_id,effect,alphafold_core_scores_residue_plddt_wt,alphafold_core_scores_protein_plddt_wt,alphafold_core_scores_protein_max_predicted_aligned_error_wt,alphafold_core_scores_proten_ptm_wt,alphafold_core_features_residue_experimentally_resolved_wt,alphafold_core_features_residue_predicted_lddt_wt,...,alphafold_core_features_residue_experimentally_resolved_mut,alphafold_core_features_residue_predicted_lddt_mut,alphafold_core_features_residue_msa_first_row_mut,alphafold_core_features_residue_single_mut,alphafold_core_features_residue_structure_module_mut,alphafold_core_features_protein_experimentally_resolved_mut,alphafold_core_features_protein_predicted_lddt_mut,alphafold_core_features_protein_msa_first_row_mut,alphafold_core_features_protein_single_mut,alphafold_core_features_protein_structure_module_mut
0,P08236,S5P,NM_000181.3:c.13T>C,Likely benign,46.591983,90.106427,31.75,0.888046,"[0.7102809, 0.71862894, 0.89787024, 0.65737355...","[-2.4777606, -2.1545868, -2.3671682, -1.591854...",...,"[0.33895195, 0.32444105, 0.43698564, 0.2935606...","[-3.6100183, -2.9942734, -2.4341316, -1.245203...","[4.7732763, 7.468932, 6.9591246, -6.897904, -7...","[7.056627, -10.289229, 55.683067, 25.23516, -2...","[0.007997297, 0.014840223, -0.006003702, 0.008...","[1.4411387, 1.5014238, 1.7631228, 1.4575887, 1...","[-9.340923, -10.505908, -9.173947, -8.192715, ...","[2.194029, 1.6112559, -3.9676542, -1.5189764, ...","[32.67436, 13.424961, 3.2069914, 1.5348411, 0....","[0.0014959556, 0.010060195, -0.0059455805, 0.0..."
1,P08236,A8S,NM_000181.3:c.22G>T,Uncertain significance,25.773571,90.106427,31.75,0.888046,"[0.2619074, 0.29129818, 0.30631134, 0.16281311...","[-2.8360972, -2.0156558, -1.8567741, -0.979897...",...,"[0.19684677, 0.22346935, 0.21782477, 0.0959438...","[-2.454352, -1.6781566, -1.5991027, -0.9269255...","[3.096415, 5.5417747, -4.2977743, 0.9525273, -...","[-0.46221876, 23.96183, 0.9521267, -43.632717,...","[0.009205885, 0.012423351, -0.0058388505, 0.00...","[1.4281887, 1.4875053, 1.7474761, 1.4440877, 1...","[-9.333698, -10.497822, -9.169257, -8.18988, -...","[2.1645148, 1.6199267, -3.9887853, -1.5169847,...","[32.72627, 13.378441, 3.1320946, 1.4491118, -0...","[0.0015348243, 0.009968462, -0.005944771, 0.00..."


3


### Save results

In [18]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold/cagi6-sherloc/shard-2000-of-4182.parquet')

In [19]:
if not DEBUG:
    pq.write_table(pa.Table.from_pandas(results_df, preserve_index=False), output_file)